In [1]:
# import the necessary libraries
import torch
from torchtext import datasets
import nltk

In [2]:
from itertools import islice

# contains full long passages from wikipedia
corpus = datasets.WikiText2(split='train')

# create one big slice of text 
corpus = ''.join(list(islice(corpus, 10000)))

# divide into words
corpus_words = corpus.split()

In [6]:
# nltk.download('punkt')
vocab = nltk.word_tokenize(corpus)
print(len(vocab))
vocab = list(set(vocab))
print(len(vocab))
# nltk.download('stopwords')
vocab = [i for i in vocab if i not in nltk.corpus.stopwords.words('english')]
print(len(vocab))

596873
23762
23619


In [ ]:
# create a mapping between words to numbers
vocab_to_int = {word: i+1 for i, word in enumerate(vocab)}
vocab_to_int["UNK"] = 0
vocab_to_int

In [9]:
# create two vectors of parameters
center = torch.rand(size=(len(vocab), 1))
context = torch.rand(size=(len(vocab), 1))

In [12]:
center[vocab_to_int['people']], context[vocab_to_int['people']]

(tensor([0.3772]), tensor([0.9885]))

In [50]:
text = ''' “'The common people pray for rain, healthy children, and a summer that never ends', Ser Jorah told her. 'It is no matter to them if the high lords play their game of thrones, so long as they are left in peace.' He gave a shrug. 'They never are.'” '''.split()

context_window = 2

for i in range(context_window, len(text[:7])-context_window):
    center_word = text[i]


    for j in range(i-context_window, i+context_window+1):
        if j == i: 
            continue
        context_word = text[j]
        print(f"P({context_word}|{center_word})", )
    print()

P(“'The|people)
P(common|people)
P(pray|people)
P(for|people)

P(common|pray)
P(people|pray)
P(for|pray)
P(rain,|pray)

P(people|for)
P(pray|for)
P(rain,|for)
P(healthy|for)

